In [164]:
import tensorflow as tf
from src.dlmpc import DeepLearningMPCController
from src.dlmpc import plant_model
from src.dlmpc import optimizer
import numpy as np

In [165]:
def multiplypredict(u,x):
    y = tf.zeros((predict_horizon,1),dtype=tf.float32)
    for i in range(predict_horizon):
        if i >= control_horizon:
            x = plant_model(x, u[-1])
        else:
            x = plant_model(x, u[i])
        y = tf.tensor_scatter_nd_update(y, [[i, 0]], x)
    return y

In [166]:
def cost_function(y_ref, predict_y, u_sequence,u0,function_type='du'):
    u0 = tf.expand_dims(u0, 1)
    if tf.shape(u0)[1] != tf.shape(u_sequence)[1]:
        u0 = tf.expand_dims(u0, 1)
    new_u0 = tf.concat((u0, u_sequence[:-1]), axis=0)
    delta_u = u_sequence - new_u0
    # 计算跟踪误差
    tracking_error = predict_y - y_ref
    # 计算成本
    # 注意：矩阵运算的顺序需要正确
    if function_type == 'du':
        cost = tf.transpose(tracking_error) @ Q @ tracking_error + tf.transpose(delta_u) @ R @ delta_u
    elif function_type == 'u':
        cost = tf.transpose(tracking_error) @ Q @ tracking_error + tf.transpose(u_sequence) @ R @ u_sequence
    else:
        cost = tf.transpose(tracking_error) @ Q @ tracking_error + tf.transpose(delta_u) @ Q @ delta_u + tf.transpose(u_sequence) @ R @ u_sequence
    return cost

In [167]:
def slove_cost_gradient(optimizer,y_ref,u_sequence,x,u0,function_type='du'):
    with tf.GradientTape() as tape:
        # 预测状态序列
        y_pred_sequence = multiplypredict(u_sequence,x)
        # 计算成本
        cost = cost_function(y_ref, y_pred_sequence, u_sequence,u0,function_type)
    # 计算梯度
    gradients = tape.gradient(cost, u_sequence)
    # 应用梯度
    optimizer.apply_gradients(zip([gradients], [u_sequence]))
    return cost, gradients,u_sequence

In [168]:
def slove_cost_jacobian(y_ref,u_sequence,x,u0,function_type='du'):
    with tf.GradientTape() as tape:
        # 预测状态序列
        y_pred_sequence = multiplypredict(u_sequence,x)
        # 计算成本
        cost = cost_function(y_ref, y_pred_sequence, u_sequence,u0,function_type)
    # 计算梯度
    jacobian = tape.jacobian(cost, u_sequence)
    return cost, jacobian,u_sequence

In [169]:
def slove_y_gradient(optimizer,u_sequence,x):
    with tf.GradientTape() as tape:
        # 预测状态序列
        y_pred_sequence = multiplypredict(u_sequence,x)
    # 计算梯度
    gradients = tape.gradient(y_pred_sequence, u_sequence)
    # 应用梯度
    optimizer.apply_gradients(zip([gradients], [u_sequence]))
    return y_pred_sequence,gradients, u_sequence

In [ ]:
@tf.function
def slove_y_jacobian(u_sequence,x):
    with tf.GradientTape() as tape:
        # 预测状态序列
        y_pred_sequence = multiplypredict(u_sequence,x)
    # 计算梯度
    jacobian = tape.jacobian(y_pred_sequence, u_sequence)
    return y_pred_sequence,jacobian, u_sequence

## 求解y对于u的梯度

In [ ]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon,1),dtype=tf.float32)
u_sequence = tf.Variable(u_sequence,trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([1],dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1,exponential_decay=True)
# 计算梯度
y_pred_sequence,gradients, u_sequence = slove_y_gradient(opt,u_sequence,x0)
print(y_pred_sequence,gradients, u_sequence)

## 求解y对于u的雅可比矩阵

In [172]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon,1),dtype=tf.float32)
u_sequence = tf.Variable(u_sequence,trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([1],dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1,exponential_decay=True)
y_pred_sequence,jacobian, u_sequence=slove_y_jacobian(u_sequence,x0)
print(y_pred_sequence,jacobian, u_sequence)

tf.Tensor(
[[1.5    ]
 [1.75   ]
 [1.875  ]
 [1.9375 ]
 [1.96875]], shape=(5, 1), dtype=float32) tf.Tensor(
[[[[2.   ]
   [0.   ]
   [0.   ]
   [0.   ]
   [0.   ]]]


 [[[1.   ]
   [2.   ]
   [0.   ]
   [0.   ]
   [0.   ]]]


 [[[0.5  ]
   [1.   ]
   [2.   ]
   [0.   ]
   [0.   ]]]


 [[[0.25 ]
   [0.5  ]
   [1.   ]
   [2.   ]
   [0.   ]]]


 [[[0.125]
   [0.25 ]
   [0.5  ]
   [1.   ]
   [2.   ]]]], shape=(5, 1, 5, 1), dtype=float32) <tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


## 方法1：求解代价函数对于u的梯度gradient

In [190]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([1], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
cost, gradients,u_sequence = slove_cost_gradient(opt,y_ref,u_sequence,x0,u0,function_type='u')
print(cost, gradients,u_sequence)

tf.Tensor([[340.833]], shape=(1, 1), dtype=float32) tf.Tensor(
[[-62.664062]
 [-59.328125]
 [-54.65625 ]
 [-46.3125  ]
 [-30.125   ]], shape=(5, 1), dtype=float32) <tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[1.1],
       [1.1],
       [1.1],
       [1.1],
       [1.1]], dtype=float32)>


## 求解代价函数对于u的雅可比矩阵，与梯度一样

In [184]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([1], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
print(slove_cost_jacobian(y_ref,u_sequence,x0,u0,function_type='u'))

(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[340.833]], dtype=float32)>, <tf.Tensor: shape=(1, 1, 5, 1), dtype=float32, numpy=
array([[[[-62.664062],
         [-59.328125],
         [-54.65625 ],
         [-46.3125  ],
         [-30.125   ]]]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>)


## 方法2：求解代价函数对于du的雅可比矩阵，再求梯度

In [187]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([[1]], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
new_u0 = tf.concat((u0, u_sequence[:-1]), axis=0)
delta_u = u_sequence - new_u0
y_pred_sequence,jacobian, u_sequence = slove_y_jacobian(u_sequence,x0)
a = np.ones((predict_horizon,predict_horizon))
for i in range(predict_horizon):
    a[i] = tf.squeeze(jacobian[i])
jacobian = tf.constant(a, dtype=tf.float32)
gradient = 2*tf.transpose(jacobian) @ (y_pred_sequence-y_ref)+2*delta_u
print(gradient)

tf.Tensor(
[[-64.66406 ]
 [-61.328125]
 [-56.65625 ]
 [-48.3125  ]
 [-32.125   ]], shape=(5, 1), dtype=float32)


## 方法2：求解代价函数对于u的雅可比矩阵，再求梯度

In [188]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([[1]], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
y_pred_sequence,jacobian, u_sequence = slove_y_jacobian(u_sequence,x0)
a = np.ones((predict_horizon,predict_horizon))
for i in range(predict_horizon):
    a[i] = tf.squeeze(jacobian[i])
jacobian = tf.constant(a, dtype=tf.float32)
gradient = 2*tf.transpose(jacobian) @ (y_pred_sequence-y_ref)+2*u_sequence
print(gradient)

tf.Tensor(
[[-62.664062]
 [-59.328125]
 [-54.65625 ]
 [-46.3125  ]
 [-30.125   ]], shape=(5, 1), dtype=float32)


In [218]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([[1]], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
y_pred_sequence,jacobian, u_sequence = slove_y_jacobian(u_sequence,x0)
a = np.ones((predict_horizon,predict_horizon))
for i in range(predict_horizon):
    a[i] = tf.squeeze(jacobian[i])
jacobian = tf.constant(a, dtype=tf.float32)
gradient = 2*tf.transpose(jacobian) @ (y_pred_sequence-y_ref)+2*u_sequence
print(gradient)

tf.Tensor(
[[-62.664062]
 [-59.328125]
 [-54.65625 ]
 [-46.3125  ]
 [-30.125   ]], shape=(5, 1), dtype=float32)


In [209]:
u_sequence-0.5*gradient

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[32.33203 ],
       [30.664062],
       [28.328125],
       [24.15625 ],
       [16.0625  ]], dtype=float32)>

In [294]:
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([[1]], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
print(u_sequence)
for j in range(50):
    y_pred_sequence,jacobian, u_sequence = slove_y_jacobian(u_sequence,x0)
    b = 1.5/(tf.norm(jacobian)**2)
    print(b)
    a = np.ones((predict_horizon,predict_horizon))
    for i in range(predict_horizon):
        a[i] = tf.squeeze(jacobian[i])
    jacobian = tf.constant(a, dtype=tf.float32)
    gradient = 2*tf.transpose(jacobian) @ (y_pred_sequence-y_ref)+1*u_sequence
    u_sequence = tf.Variable(u_sequence - b*gradient,trainable=True)
    x0 = y_pred_sequence[0].numpy()
    print(u_sequence,gradient)

<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>
tf.Tensor(0.060263652, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[4.836629 ],
       [4.6355934],
       [4.3540487],
       [3.851224 ],
       [2.8757062]], dtype=float32)> tf.Tensor(
[[-63.664062]
 [-60.328125]
 [-55.65625 ]
 [-47.3125  ]
 [-31.125   ]], shape=(5, 1), dtype=float32)
tf.Tensor(0.0033665039, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[2.4685683],
       [1.87763  ],
       [1.9505422],
       [2.2818737],
       [2.2998714]], dtype=float32)> tf.Tensor(
[[703.41833]
 [819.2367 ]
 [713.9473 ]
 [466.16614]
 [171.04828]], shape=(5, 1), dtype=float32)
tf.Tensor(0.012672722, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[1.2415696],
       [1.5892069],
       [1.8917757],
       [2.198438

In [295]:
y_pred_sequence

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[9.874999],
       [9.875002],
       [9.875   ],
       [9.875002],
       [9.750001]], dtype=float32)>

In [197]:
2/tf.norm(jacobian)

<tf.Tensor: shape=(), dtype=float32, numpy=0.4008779>

In [416]:
predict_horizon = 5
control_horizon = 1
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([[1]], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
print(u_sequence)
for j in range(50):
    y_pred_sequence,jacobian, u_sequence = slove_y_jacobian(u_sequence,x0)
    new_u0 = tf.concat((u0, u_sequence[:-1]), axis=0)
    delta_u = u_sequence - new_u0
    # a = np.ones((predict_horizon,predict_horizon))
    # for i in range(predict_horizon):
    #     a[i] = tf.squeeze(jacobian[i])
    # jacobian = tf.constant(a, dtype=tf.float32)
    jacobian = tf.reshape(jacobian, [-1,control_horizon])
    b = 0.5/(tf.transpose(jacobian)@jacobian)
    # b = 1.0/(tf.norm(jacobian,ord=2)**2)
    print(b)
    gradient = 2*tf.transpose(jacobian) @ (y_pred_sequence-y_ref)+2*delta_u
    u_sequence = tf.clip_by_value(u_sequence - b*gradient, 0, 3)
    u_sequence = tf.Variable(u_sequence,trainable=True)
    x0 = y_pred_sequence[0].numpy()
    print(u_sequence,gradient)

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[1.]], dtype=float32)>
tf.Tensor([[0.00920334]], shape=(1, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[3.]], dtype=float32)> tf.Tensor([[-263.08594]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.00102259]], shape=(1, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[2.4618645]], dtype=float32)> tf.Tensor([[526.2461]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.00151851]], shape=(1, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[2.2467365]], dtype=float32)> tf.Tensor([[141.67068]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.00182323]], shape=(1, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[2.2120516]], dtype=float32)> tf.Tensor([[19.023897]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.00188085]], shape=(1, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 1)

In [417]:
y_pred_sequence

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[9.965767],
       [9.965767],
       [9.965767],
       [9.965767],
       [9.965767]], dtype=float32)>

In [420]:
b = 0.5/(tf.transpose(jacobian)@jacobian)
# b = 0.5/(tf.norm(jacobian,ord=2)**2)
print(b)

tf.Tensor([[0.00184699]], shape=(1, 1), dtype=float32)


In [421]:
1/tf.norm(tf.transpose(gradient)@gradient)

<tf.Tensor: shape=(), dtype=float32, numpy=472286140.0>

In [296]:
jacobian

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[4.444097  , 0.        , 0.        , 0.        , 0.        ],
       [2.2220485 , 4.4440985 , 0.        , 0.        , 0.        ],
       [1.1110243 , 2.2220492 , 4.4440966 , 0.        , 0.        ],
       [0.55551213, 1.1110246 , 2.2220483 , 4.444098  , 0.        ],
       [0.27775607, 0.5555123 , 1.1110241 , 2.222049  , 4.387482  ]],
      dtype=float32)>

In [297]:
tf.transpose(jacobian)@jacobian

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[26.322006 , 13.122408 ,  6.4840136,  3.0876255,  1.218771 ],
       [13.122408 , 26.244816 , 12.968027 ,  6.175251 ,  2.437542 ],
       [ 6.4840136, 12.968027 , 25.936054 , 12.350502 ,  4.875084 ],
       [ 3.0876255,  6.175251 , 12.350502 , 24.701004 ,  9.750168 ],
       [ 1.218771 ,  2.437542 ,  4.875084 ,  9.750168 , 19.243301 ]],
      dtype=float32)>

In [306]:
tf.sqrt(tf.reduce_sum(jacobian*jacobian))

<tf.Tensor: shape=(), dtype=float32, numpy=11.063339>

In [301]:
import tensorflow as tf

# 创建一个矩阵
matrix = tf.constant([[1.0, 2.0], [3.0, 4.0]])

# 计算矩阵的二范数（Frobenius范数）
fro_norm = tf.norm(matrix, ord=2)

# 输出结果
print("Frobenius范数:", fro_norm)


Frobenius范数: tf.Tensor(5.4772253, shape=(), dtype=float32)


In [302]:
tf.norm(jacobian, ord=2)

<tf.Tensor: shape=(), dtype=float32, numpy=11.063339>

In [317]:
1.5/(tf.norm(jacobian,ord=2)**2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0122551555>

In [312]:
1.5/tf.norm(tf.transpose(jacobian)@jacobian,ord=2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.02247963>

In [313]:
1.5/tf.norm(jacobian * jacobian,ord=2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0332669>

In [315]:
1.5/tf.sqrt(tf.reduce_sum(jacobian*jacobian))

<tf.Tensor: shape=(), dtype=float32, numpy=0.13558294>

In [379]:
predict_horizon = 5
control_horizon = 1
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([[1]], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
print(u_sequence)
n =1
c = 2
for j in range(50):
    y_pred_sequence,jacobian, u_sequence = slove_y_jacobian(u_sequence,x0)
    b = 1.5/(tf.norm(jacobian,ord=2)**2)
    print(b)
    new_u0 = tf.concat((u0, u_sequence[:-1]), axis=0)
    delta_u = u_sequence - new_u0
    y_pred_sequence,jacobian, u_sequence = slove_y_jacobian(u_sequence,x0)
    # a = np.ones((predict_horizon,predict_horizon))
    # for i in range(predict_horizon):
    #     a[i] = tf.squeeze(jacobian[i])
    # jacobian = tf.constant(a, dtype=tf.float32)
    jacobian = tf.reshape(jacobian, [-1,control_horizon])
    gradient = tf.transpose(jacobian) @ (y_pred_sequence-y_ref)
    n = n * np.e**(-c*i)
    c= c*(1-j*0.01)
    u_sequence = tf.Variable(u_sequence - 1*n/(1+n)*gradient,trainable=True)
    x0 = y_pred_sequence[0].numpy()
    print(u_sequence,gradient)

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[1.]], dtype=float32)>
tf.Tensor(0.02761001, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[1.0441129]], dtype=float32)> tf.Tensor([[-131.54297]], shape=(1, 1), dtype=float32)


In [378]:
jacobian

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[2.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 2.   , 0.   , 0.   , 0.   ],
       [0.5  , 1.   , 2.   , 0.   , 0.   ],
       [0.25 , 0.5  , 1.   , 2.   , 0.   ],
       [0.125, 0.25 , 0.5  , 1.   , 2.   ]], dtype=float32)>

In [370]:
1.5 / tf.norm(jacobian * jacobian, ord=2)
1.5 / tf.sqrt(tf.reduce_sum(jacobian * jacobian))
predict_horizon = 5
control_horizon = 5
Q = tf.eye(predict_horizon)
R = tf.eye(predict_horizon)
u_sequence = tf.ones((control_horizon, 1), dtype=tf.float32)
u_sequence = tf.Variable(u_sequence, trainable=True)
y_ref = 10
x0 = 1
u0 = tf.constant([[1]], dtype=tf.float32)
opt = optimizer('sgd', 0.1, 1, exponential_decay=True)
print(u_sequence)
n = 1
c = 2
for j in range(1):
    y_pred_sequence, jacobian, u_sequence = slove_y_jacobian(u_sequence, x0)
    b = 1.5 / (tf.norm(jacobian, ord=2) ** 2)
    # print(b)
    new_u0 = tf.concat((u0, u_sequence[:-1]), axis=0)
    delta_u = u_sequence - new_u0
    y_pred_sequence, jacobian, u_sequence = slove_y_jacobian(u_sequence, x0)
    print(tf.reshape(jacobian, [-1,control_horizon]))

<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>
tf.Tensor(
[[2.    0.    0.    0.    0.   ]
 [1.    2.    0.    0.    0.   ]
 [0.5   1.    2.    0.    0.   ]
 [0.25  0.5   1.    2.    0.   ]
 [0.125 0.25  0.5   1.    2.   ]], shape=(5, 5), dtype=float32)


In [367]:
print(tf.reshape(jacobian, [-1,control_horizon]))

tf.Tensor(
[[2.    0.   ]
 [1.    2.   ]
 [0.5   3.   ]
 [0.25  3.5  ]
 [0.125 3.75 ]], shape=(5, 2), dtype=float32)


In [363]:
tf.reshape(jacobian, [-1,2])

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[2.   , 0.   ],
       [1.   , 2.   ],
       [0.5  , 3.   ],
       [0.25 , 3.5  ],
       [0.125, 3.75 ]], dtype=float32)>

In [422]:
y_pred_sequence

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[9.965767],
       [9.965767],
       [9.965767],
       [9.965767],
       [9.965767]], dtype=float32)>

In [423]:
a = 1
for i in range(10):
    a = a * np.e**(-i)
    print(a)

1.0
0.36787944117144233
0.04978706836786395
0.0024787521766663594
4.539992976248488e-05
3.0590232050182605e-07
7.582560427911916e-10
6.914400106940214e-13
2.319522830243574e-16
2.862518580549401e-20


In [331]:

print(np.e)

2.718281828459045
